In [1]:
import pandas as pd
import numpy as np
import os
import sys
import math
import random

from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import warnings
from sklearn import preprocessing
import sklearn.preprocessing
from gensim import corpora, models, similarities

unable to import 'smart_open.gcs', disabling that module
F:\anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [4]:
def getWord_model(word,num_features,min_count, model, Unfile):
	word_model = ""
	if not os.path.isfile(model):
		sentence = LineSentence(Unfile,max_sentence_length = 15000)

		num_features = int(num_features)
		min_word_count = int(min_count)
		num_workers = 20
		context = 20
		downsampling = 1e-3

		print ("Training Word2Vec model...")
		word_model = Word2Vec(sentence, workers=num_workers,\
						size=num_features, min_count=min_word_count, \
						window=context, sample=downsampling, seed=1,iter = 50)
		word_model.init_sims(replace=False)
		word_model.save(model)

	else:
		print ("Loading Word2Vec model...")
		word_model = Word2Vec.load(model)

	return word_model

def DNAToWord(dna, K):

	sentence = ""
	length = len(dna)

	for i in range(length - K + 1):
		sentence += dna[i: i + K] + " "

	sentence = sentence[0 : len(sentence) - 1]
	return sentence


def getDNA_split(DNAdata,word):

	list1 = []
	list2 = []
	for DNA in DNAdata["seq1"]:
		DNA = str(DNA).upper()
		list1.append(DNAToWord(DNA,word).split(" "))

	for DNA in DNAdata["seq2"]:
		DNA = str(DNA).upper()
		list2.append(DNAToWord(DNA,word).split(" "))

	return list1,list2

def getAvgFeatureVecs(DNAdata1,DNAdata2,model,num_features):
	counter = 0
	DNAFeatureVecs = np.zeros((len(DNAdata1),2*num_features), dtype="float32")
	
	for DNA in DNAdata1:
		if counter % 1000 == 0:
			print ("DNA %d of %d\r" % (counter, len(DNAdata1)))
			sys.stdout.flush()

		DNAFeatureVecs[counter][0:num_features] = np.mean(model[DNA],axis = 0)
		counter += 1
	print()
	
	counter = 0
	for DNA in DNAdata2:
		if counter % 1000 == 0:
			print ("DNA %d of %d\r" % (counter, len(DNAdata2)))
			sys.stdout.flush()
		DNAFeatureVecs[counter][num_features:2*num_features] = np.mean(model[DNA],axis = 0)
		counter += 1

	return DNAFeatureVecs

def npyTosvm(npyfile, svmfile, pos_num):
	dataDataVecs = np.load(npyfile)
	g = open(svmfile,'w')
	print(len(dataDataVecs))
	#print(dataDataVecs[0])
	m = 0
	for i in range(len(dataDataVecs)):
		line = ''
		for j in range(len(dataDataVecs[0])):
			if j == len(dataDataVecs[0])-1:
				line += str(j+1)+':'+str(dataDataVecs[i][j])+'\n'
			else:
				line += str(j+1)+':'+str(dataDataVecs[i][j])+'\t'
		m += 1
		if m < (pos_num+1):
			g.write('1\t'+line)
		else:
			g.write('0\t'+line)

def SVMtoCSV(svmfile, csvfile):
	f = open(svmfile,'r')
	g = open(csvfile,'w')
	lines = f.readlines()
	legth = len(lines[0].split('	'))-1
	#print(legth)
	classline = 'class'
	for i in range(legth):
		classline += ',%d'%(i+1)
	g.write(classline+'\n')

	for line in lines:
		line = line.strip('\n').split('	')
		g.write(line[0]+',')

		legth2 = len(line[1:])
		m = 0
		for j in line[1:]:
			if m == legth2-1:
				j = j.split(':')[-1]
				g.write(j)
				m += 1
			else:
				j = j.split(':')[-1]
				g.write(j+',')
				m += 1
		g.write('\n')

	f.close()
	g.close()

In [6]:
kmer = 3
seqfile = 'test.fa'
pos_number = 10 # NOTE: the number of postive sample in test file

#### generate Unsupervised ##### 
Unfile = '%dUn'%(kmer)
g = open(Unfile,'w')
DNAseq = pd.read_csv(seqfile,sep = "\t",error_bad_lines=False)
words1,words2 = getDNA_split(DNAseq,kmer)

for i in range(len(words1)):
	line = ' '.join(words1[i])
	g.write(line+'\n')

for i in range(len(words2)):
	line = ' '.join(words2[i])
	g.write(line+'\n')
g.close()

#####get word2vec model#####
model = 'model_%d'%(kmer)
fea_num = 100
min_fea = 10
getWord_model(kmer,fea_num,min_fea,model,Unfile)

####obtain word2vec feature set####

word_model = Word2Vec.load(model)
dataDataVecs = getAvgFeatureVecs(words1,words2,word_model,fea_num)
print (dataDataVecs.shape)
fea_npy = '%d_vecs.npy'%(kmer)
np.save(fea_npy,dataDataVecs)


#### npy To csv #####
fea_svm = '%d_vecs.svm'%(kmer)
fea_csv = '%d_vecs.csv'%(kmer)

npyTosvm(fea_npy, fea_svm,pos_number)
SVMtoCSV(fea_svm, fea_csv)


Training Word2Vec model...
DNA 0 of 20

DNA 0 of 20
(20, 200)
20


F:\anaconda\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
F:\anaconda\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:67: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
